In [13]:
pip install "ray[default]"

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install -U ray[tune] tensorboardx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Using cached protobuf-4.23.4-cp37-abi3-manylinux2014_x86_64.whl (304 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.154.0 requires protobuf<4.0,>=3.1, but you have protobuf 4.23.4 which is incompatible.
sagemaker-training 4.5.0 requires protobuf<=3.20.3,>=3.9.2, but you have protobuf 4.23.4 which is incompatible.
smdebug 1.0.34 requires protobuf<=3.20.3,>=3.20.0, but you have protobuf 4.23.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [16]:
import torch
import tensorflow
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.parallel
import ray
from ray import tune
from ray.tune import Callback
from ray.tune import grid_search, uniform
from ray.tune.integration.keras import TuneReportCallback
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.examples.utils import get_iris_data
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import argparse
import scipy.io
import os

In [17]:
mat_data = scipy.io.loadmat('Porsche/7000_scaled_2LVL_properties_without_Udc_nocycle_data.mat')


#Get scaled input and scaled output data
#Min max Scaling
input_scaled = mat_data.get('input_scaled') #(4500000, 8)
output_scaled = mat_data.get('output_scaled') #(4500000, 4)

#Z-Score scaling
#input_scaled = mat_data.get('input_standardized') #(4500000, 8)
#output_scaled = mat_data.get('output_standardized') #(4500000, 4)

# Split the dataset into input and output subsets
input_subset = input_scaled
output_subset = output_scaled
# Verify the shape of the resulting subsets
print(input_subset.shape)
print(output_subset.shape)

(2033336, 35)
(2033336, 31)


In [18]:
class MLP(nn.Module):
  def __init__(self, input_size, output_size, activation_func, dropout = 0.1, num_layers = 3, num_neurons = 2048):
        super(MLP, self).__init__()
        self.num_layers = num_layers
        self.layers = nn.ModuleList()
        self.bn_layers = nn.ModuleList()
        #self.activation_fn = nn.ReLu()
        self.activation_fn = activation_func
        self.sigmoid = nn.Sigmoid()

        for i in range(num_layers):
          if i == 0:
            self.layers.append(nn.Linear(input_size, num_neurons))
            self.bn_layers.append(nn.BatchNorm1d(num_neurons))
          else:
            self.layers.append(nn.Linear(num_neurons // (2**(i-1)), num_neurons // (2**i)))
            self.bn_layers.append(nn.BatchNorm1d(num_neurons // (2**i)))

        # Try Relu also here after going through hyperparamter tuning
        self.output_layer = nn.Linear(num_neurons // (2**(num_layers-1)), output_size)
        self.dropout = nn.Dropout(dropout)

  def forward(self, x):
      for i in range(self.num_layers):
        x = self.layers[i](x)
        x = self.bn_layers[i](x)
        x = self.dropout(x)
        x = self.activation_fn(x)

      x = self.output_layer(x)
      x = self.sigmoid(x)
      return x

In [19]:
# Set device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
#Set data and target
# Remove the label column from the input data
#input_data = torch.Tensor(input_subset[:, :-1])  # Exclude the last column, assuming it contains labels
#output_data = torch.Tensor(output_subset[:, :-1])  # Include all columns, assuming it contains target values
#With label

input_data = torch.Tensor(input_subset)
output_data = torch.Tensor(output_subset)
  #print(input_data)
# Define the get_iris_data function
def get_iris_data():


  # Split the data into training, validation, and testing sets
  #test_size = 0.2 ist 80% Training set
  train_data, test_data, train_target, test_target = train_test_split(
      input_data, output_data, test_size=0.3, random_state=42
  )

  #test_size hier auf 0.5 heißt die übrigen 20% nochmal in 10% für val und 10% für test
  val_data, test_data, val_target, test_target = train_test_split(
      test_data, test_target, test_size=0.5, random_state=42
  )
  #data = pd.DataFrame(train_data.numpy())
  #print(len(data.iloc[:,7]))
  #unique_values = data.iloc[:,7].unique()
  #print(len(unique_values))
  # Create data loaders
  train_dataset = TensorDataset(train_data, train_target)
  train_loader = DataLoader(train_dataset, batch_size = 64, shuffle=True)
  val_dataset = TensorDataset(val_data, val_target)
  val_loader = DataLoader(val_dataset, batch_size = 64, shuffle=False)
  test_dataset = TensorDataset(test_data, test_target)
  test_loader = DataLoader(test_dataset, batch_size = 64, shuffle=False)

  return train_loader, val_loader, test_loader

cuda


In [20]:
# Define the dimensions of your input, hidden, and output layers
input_size = input_data.shape[1]
output_size = output_data.shape[1]

#Define inital data
test_accuracies = []
best_test_accuracy = 0.0
best_val_loss = float('inf')
train_losses = []
val_losses = []

num_epochs = 1

# Define your data loaders here using the current batch_size
train_loader, val_loader, test_loader = get_iris_data()
# Serialize the necessary components of the data loaders
#train_loader_serialized = (train_loader.dataset,)
#val_loader_serialized = (val_loader.dataset,)

In [21]:
#pip install --upgrade ray[tune]
# Check Ray Tune version
os.environ["TUNE_DISABLE_STRICT_METRIC_CHECKING"] = "1"

In [22]:
from sklearn.metrics import r2_score
import sys
import gc
import psutil 

def train_model(config, num_epochs, input_size, output_size, checkpoint_dir = None):

    if checkpoint_dir is None:
        checkpoint_dir = 'Porsche/checkpoints'

    # Initialize with a high value
    best_val_loss = float('inf')
    best_r2score = float('-inf')
    # Generate a unique trial ID for the checkpoint file
    trial_id = tune.get_trial_id()
    #best_checkpoint_path = os.path.join(checkpoint_dir, "best_checkpoint.pt")
    best_checkpoint_path = os.path.join(checkpoint_dir, f"{trial_id}_best_checkpoint.pt")

    # Extract the hyperparameters from the config
    activation_func = activation_func_mapping[config["activation_func"]]
    optimizer_name = config["optimizer"]
    #dropout = config["dropout"]
    lr=config["lr"]

    # Create and compile your model using the provided hyperparameters
    model = MLP(input_size, output_size, activation_func)
    model.to(device)

    criterion = nn.MSELoss()
    optimizer = optimizer_mapping[optimizer_name](model.parameters(), lr=config["lr"])
    #optimizer = optim.Adamax(model.parameters(), lr)

    # Continue with the training code
    for epoch in range(num_epochs):
        train_loss = 0.0
        # Training
        model.train()
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            predictions = []
            targets = []
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                predictions.append(outputs.cpu().numpy())
                targets.append(labels.cpu().numpy())

            val_loss /= len(val_loader)
            val_losses.append(val_loss)

        predictions = np.concatenate(predictions, axis=0)
        targets = np.concatenate(targets, axis=0)
        r2score = r2_score(targets, predictions)
        # Report the metric to Tune
        tune.report(r2score=r2score, val_loss=val_loss, _metric = r2score)

        # Check if the current r2score is the best so far
        if r2score > best_r2score:
          best_r2score = r2score
          # Save the checkpoint of the best model
          with tune.checkpoint_dir(step=epoch) as checkpoint_dir:
            torch.save({"state_dict": model.state_dict(),"config": config}, best_checkpoint_path)
    return best_r2score

ray.shutdown()
ray.init()
checkpoint_dir = 'Porsche/checkpoints'

config = {
    "lr": tune.grid_search([0.01]),
    "optimizer": tune.grid_search(['Adam', 'Adamax']),
    "activation_func": tune.grid_search(["Tanh"])
}
optimizer_mapping = {
"Adam": optim.Adam,
"Adamax": optim.Adamax
}

activation_func_mapping = {
"ReLU": nn.ReLU(),
"Tanh": nn.Tanh()
}

# Define the fixed parameters using tune.with_parameters
train_model_with_params = tune.with_parameters(
    train_model,
    train_loader= train_loader,
    val_loader= val_loader,
    num_epochs=num_epochs,
    input_size=input_size,
    output_size=output_size,
    checkpoint_dir = checkpoint_dir #/content
)


analysis = tune.run(
    train_model_with_params,
    config = config,
    metric="r2score",
    mode="max",
    resources_per_trial={"gpu": 1},  # Specify 1 GPU per trial
    verbose=1,
    checkpoint_score_attr='r2score',
    progress_reporter=CLIReporter(metric_columns=["r2score"]),
    local_dir=checkpoint_dir  # Specify the checkpoint directory
)


best_trial = analysis.get_best_trial(metric="r2score", mode="max")
best_params = best_trial.config
best_metric = best_trial.metric_analysis["r2score"]["max"]

# Manually retrieve the best checkpoint path using the Ray Checkpoint module
print(best_trial.trial_id)
print(best_params)
print(best_metric)

2023-07-18 17:15:16,392	WARNING services.py:1826 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 1901010944 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=2.69gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-07-18 17:15:16,691	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
/opt/conda/lib/python3.10/site-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the`RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(


ValueError: Tracked actor is not managed by this event manager: <TrackedActor 251618162257320281813497558333759770004>